# Random Forest Classifier

In [190]:
import pandas as pd
import numpy as np

freq = 'Daily'
stock = 'AAPL'
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.shift(-1).values[:-1]
predictors = pd.read_csv('../predictors/Tech/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values[:-1]
predictors

,SMA,EMA,STOCH_k,STOCK_d,RSI,MFI,SAR,AD,MACD,MACD_Signal,MACD_Histo,VWAP,SPY,NDAQ
Date,,,,,,,,,,,,,,
2010-01-04,202.878000,205.345351,21392.872340,21204.475820,75.356618,80.785184,1.148300e+05,4.327271e+10,3.926500,2.148400,1.778100,135.320845,88.050132,5.597397
2010-01-05,204.120857,206.549971,21430.232615,21295.994172,74.035805,81.246629,1.171224e+05,4.451263e+10,4.287593,2.576239,1.711354,135.391125,88.283213,5.613990
2010-01-06,205.320857,207.139308,21089.232615,21304.112523,72.828568,79.144477,1.194605e+05,4.329049e+10,4.249616,2.910914,1.338702,135.453784,88.345367,5.611225
2010-01-07,206.431571,207.598067,21050.232615,21189.899282,71.403403,72.445029,1.218455e+05,4.206772e+10,4.140322,3.156796,0.983526,135.506599,88.718297,5.536556
2010-01-08,207.868714,208.182325,21189.622445,21109.695891,79.112166,78.095086,1.242783e+05,4.330626e+10,4.119191,3.349275,0.769916,135.556389,89.013532,5.594632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-27,312.660714,310.922870,30883.645013,31539.108233,59.513412,63.803522,6.836812e+26,3.970456e+11,9.595247,10.551674,-0.956427,195.714203,307.043805,34.929343
2020-01-28,314.039286,311.825154,31757.645013,31487.572259,67.412486,64.905487,6.973548e+26,4.005847e+11,9.322036,10.305746,-0.983710,195.764709,310.261359,35.362062
2020-01-29,315.550000,313.493800,32420.727035,31687.339021,68.465165,66.624864,7.113019e+26,4.041193e+11,9.532232,10.151043,-0.618811,195.836793,310.005094,36.698400


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,zero_one_loss,recall_score
import sklearn
print(sklearn.__version__)
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,shuffle=False)
sm = SMOTE(sampling_strategy='minority',random_state=42,k_neighbors=5)
X_res,y_res = sm.fit_resample(X_train,y_train)
pipe = make_pipeline(StandardScaler(),RandomForestClassifier(criterion='log_loss'))
pipe.fit(X_res,y_res)

y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
model = XGBClassifier(base_score=0.5, booster='gbtree',objective='binary:logistic', learning_rate=0.05, max_depth=7,n_estimators=100)
model = make_pipeline(StandardScaler(),model)
model.fit(X_res,y_res)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'learning_rate' : [0.05, 0.10, 0.20,0.15],  
          'min_child_weight': [1, 5, 10,15],  
          'gamma': [0.5, 1, 5,0.1],  
          'subsample': [0.6, 0.8, 1.0,0.9],  
          'colsample_bytree': [0.6, 0.8, 1.0,0.9],  
           'max_depth': [ 4, 5,6,7],
           'n_estimators':[100,200,300,400]
         }  
grid = GridSearchCV(xgb.XGBClassifier()
                   ,param_grid
                   ,n_jobs=1
                   ,cv=4
                   ,verbose=1
                   ,refit=True)
grid.fit(X_res, y_res)


In [ ]:
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(model.feature_importances_)
plt.figure(figsize=(15,8))
plt.bar(range(len(model.feature_importances_)), model.feature_importances_)
plt.title('Feature importance')
plt.xticks(range(len(model.feature_importances_)),predictors.columns,rotation=60)
plt.show()

In [ ]:
important_features = predictors.columns[np.where(model.feature_importances_>0.025)]

In [ ]:
X_se = predictors[important_features].values[:-1]
X_train,X_test,y_train,y_test = train_test_split(X_se,y,test_size=0.3,shuffle=False)
sm = SMOTE(sampling_strategy='minority',random_state=42,k_neighbors=5)
X_res,y_res = sm.fit_resample(X_train,y_train)
model = XGBClassifier(base_score=0.5, booster='gbtree',objective='binary:logistic', learning_rate=0.1, max_depth=3,n_estimators=500)
model.fit(X_res,y_res)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
X_se = predictors[important_features].values[:-1]
X_train,X_test,y_train,y_test = train_test_split(X_se,y,test_size=0.3,shuffle=False)
sm = SMOTE(sampling_strategy='minority',random_state=42,k_neighbors=5)
X_res,y_res = sm.fit_resample(X_train,y_train)
model = XGBClassifier(base_score=0.5, booster='gbtree',objective='binary:logistic', learning_rate=0.1, max_depth=3,n_estimators=500)
model.fit(X_res,y_res)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
pipe = make_pipeline(StandardScaler(),RandomForestClassifier(criterion='log_loss'))
pipe.fit(X_res,y_res)
y_pred = pipe.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
dtrain = xgb.DMatrix(data=X_train, label=y_train, enable_categorical = True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)
num_parallel_tree = 4
num_boost_round = 16
# total number of built trees is num_parallel_tree * num_classes * num_boost_round

# We build a boosted random forest for classification here.
booster = xgb.train({
    'num_parallel_tree': 4, 'subsample': 0.5, 'num_class': 2},
                    num_boost_round=num_boost_round, dtrain=dtrain)
    


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
def logi_cv(stock, freq, cv=5):
    # freq = 'Weekly'
    # stock = 'AAPL'
    price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
    y = price.direction.shift(-1).values[:-1]
    predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
    predictors.fillna(0,inplace=True)
    X = predictors.values[:-1,]
    X.shape

    tscv = TimeSeriesSplit(n_splits=cv)
    scores = []
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pipe = make_pipeline(MinMaxScaler(),RandomForestClassifier(criterion='log_loss'))
        pipe.fit(X_train,y_train)
        y_pred = pipe.predict(X_test)
        score = classification_report(y_test,y_pred,output_dict=True)
        scores.append(score['weighted avg']['f1-score'])

    average_score = np.mean(scores)
    print(stock,'/',freq,'\n','Average F1-score:', average_score)

In [ ]:
logi_cv("AAPL","Daily")

#

In [ ]:
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.shift(-1).values[:-1]
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values[:-1,]
X.shape
tscv = TimeSeriesSplit(n_splits=cv)
scores = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pipe = make_pipeline(MinMaxScaler(),RandomForestClassifier(criterion='log_loss'))
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    score = classification_report(y_test,y_pred,output_dict=True)
    scores.append(score['weighted avg']['f1-score'])
average_score = np.mean(scores)
print(stock,'/',freq,'\n','Average F1-score:', average_score)

In [194]:
logi_cv("AAPL","Daily")

AAPL / Daily 
 Average F1-score: 0.44706501401326026


#

In [ ]:
price = pd.read_csv('../encode_price/'+freq+'/'+stock+'.csv')
y = price.direction.shift(-1).values[:-1]
predictors = pd.read_csv('../predictors/Merged/'+freq+'/'+stock+'.csv',index_col='Date')
predictors.fillna(0,inplace=True)
X = predictors.values[:-1,]
X.shape
tscv = TimeSeriesSplit(n_splits=cv)
scores = []
for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    pipe = make_pipeline(MinMaxScaler(),RandomForestClassifier(criterion='log_loss'))
    pipe.fit(X_train,y_train)
    y_pred = pipe.predict(X_test)
    score = classification_report(y_test,y_pred,output_dict=True)
    scores.append(score['weighted avg']['f1-score'])
average_score = np.mean(scores)
print(stock,'/',freq,'\n','Average F1-score:', average_score)